In [5]:
%pip install --upgrade pip
%pip install amazon-braket-sdk==1.37.0

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.6 MB/s eta 0:00:0000:01
  Attempting uninstall: amazon-braket-sdk
    Found existing installation: amazon-braket-sdk 1.38.0
    Uninstalling amazon-braket-sdk-1.38.0:
      Successfully uninstalled amazon-braket-sdk-1.38.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --upgrade amazon-braket-schemas
!qbraid jobs disable qbraid_sdk

Note: you may need to restart the kernel to use updated packages.
 You have already disabled qBraid Quantum Jobs in the qbraid_sdk environment.


In [1]:
import cirq
import qbraid
from qbraid import interface
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

def prep(sys,state):
    for i in range(len(state)):
        if state[i]=='1':
            yield cirq.X(sys[i])

def zz_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[0])
    yield cirq.Rz(rads=np.arccos(1-2*p)).on(sys[0]).controlled_by(env[0])
    yield cirq.H(sys[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[1])

def xx_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.H(sys[1])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[1],env[0])
    yield cirq.H(sys[1])
    yield cirq.Rz(rads=np.arccos(1-2*p)).on(sys[1]).controlled_by(env[0])
    yield cirq.H(sys[1])
    yield cirq.CNOT(sys[1],env[0])

def zz_xx_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[0])
    yield cirq.Rz(rads=np.arccos(1-2*p)).on(sys[0]).controlled_by(env[0])
    yield cirq.H(sys[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[1])
    yield cirq.X(env[1])
    yield cirq.CNOT(sys[1],env[1])
    yield cirq.H(sys[1])
    yield cirq.Rz(rads=np.arccos(1-2*p)).on(sys[1]).controlled_by(env[1])
    yield cirq.H(sys[1])
    yield cirq.CNOT(sys[1],env[1])


In [2]:
probs = np.linspace(0,1,num=5)
repcnt = 960

In [4]:
!qbraid jobs enable qbraid_sdk

You have already enabled qBraid Quantum Jobs in the qbraid_sdk environment.


In [1]:
from qbraid import get_devices
get_devices()

Provider,Name,qBraid ID,Status
AWS,DM1,aws_dm_sim,●
AWS,SV1,aws_sv_sim,●
AWS,TN1,aws_tn_sim,●
IBM,Belem,ibm_q_belem,●
IBM,Ext. stabilizer simulator,ibm_q_simulator_extended_stabilizer,●
IBM,Jakarta,ibm_q_jakarta,●
IBM,Lagos,ibm_q_lagos,○
IBM,Lima,ibm_q_lima,●
IBM,MPS simulator,ibm_q_simulator_mps,●
IBM,Manila,ibm_q_manila,●


In [3]:
from qbraid import device_wrapper, job_wrapper, get_jobs
ionq_id = "aws_ionq_aria1"
qbraid_aws_device = device_wrapper(ionq_id)
qbraid_aws_device.vendor_dlo
qbraid_aws_device.info

{'numberQubits': 11,
 'objRef': None,
 'pendingJobs': 'N/A',
 'availabilityCD': '04:02:14',
 'visibility': 'public',
 'qbraid_id': 'aws_ionq',
 'name': 'IonQ Device',
 'provider': 'IonQ',
 'paradigm': 'gate-based',
 'type': 'QPU',
 'dateDeployed': '2019',
 'location': 'College Park, MD, USA',
 'vendor': 'AWS',
 'runPackage': 'braket',
 'status': 'OFFLINE',
 'createdAt': '2022-03-10T04:08:21.014Z',
 'updatedAt': '2023-05-12T08:57:47.069Z',
 '__v': 0,
 'architecture': 'trapped ion'}

In [ ]:
#circ_r = transpile(circ,basis_gates=['measure', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cx', 'swap'])
    
    from qbraid import circuit_wrapper
    circ_wrap = circuit_wrapper(circ)
    circ_fin = circ_wrap.transpile("braket")
    return circ_fin
 
    #qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    #return qbraid_aws_job

In [4]:
def batch_1():
    env = [cirq.LineQubit(14),cirq.LineQubit(15),cirq.LineQubit(16),cirq.LineQubit(17),cirq.LineQubit(18),\
           cirq.LineQubit(19),cirq.LineQubit(20),cirq.LineQubit(21),cirq.LineQubit(22),cirq.LineQubit(23),\
           cirq.LineQubit(24)]
    sys = [cirq.LineQubit(0),cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(3),cirq.LineQubit(4),cirq.LineQubit(5),\
          cirq.LineQubit(6),cirq.LineQubit(7),cirq.LineQubit(8),cirq.LineQubit(9),cirq.LineQubit(10),cirq.LineQubit(11),\
          cirq.LineQubit(12),cirq.LineQubit(13)]
    circ = cirq.Circuit()
    
    circ.append(zz_xx_pump(sys[0:2],env[0:2],probs[0],'00'))
    circ.append(zz_xx_pump(sys[2:4],env[2:4],probs[0],'01'))
    circ.append(zz_xx_pump(sys[4:6],env[4:6],probs[0],'10'))
    circ.append(zz_xx_pump(sys[6:8],env[6:8],probs[0],'11'))
    circ.append(zz_pump(sys[8:10],env[8:9],probs[0],'00'))
    circ.append(zz_pump(sys[10:12],env[9:10],probs[0],'01'))
    circ.append(zz_pump(sys[12:14],env[10:11],probs[0],'10'))
 
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job

In [5]:
def batch_2():
    env = [cirq.LineQubit(14),cirq.LineQubit(15),cirq.LineQubit(16),cirq.LineQubit(17),cirq.LineQubit(18),\
           cirq.LineQubit(19),cirq.LineQubit(20),cirq.LineQubit(21),cirq.LineQubit(22),cirq.LineQubit(23),\
           cirq.LineQubit(24)]
    sys = [cirq.LineQubit(0),cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(3),cirq.LineQubit(4),cirq.LineQubit(5),\
          cirq.LineQubit(6),cirq.LineQubit(7),cirq.LineQubit(8),cirq.LineQubit(9),cirq.LineQubit(10),cirq.LineQubit(11),\
          cirq.LineQubit(12),cirq.LineQubit(13)]
    circ = cirq.Circuit()
    
    circ.append(zz_xx_pump(sys[0:2],env[0:2],probs[1],'00'))
    circ.append(zz_xx_pump(sys[2:4],env[2:4],probs[1],'01'))
    circ.append(zz_xx_pump(sys[4:6],env[4:6],probs[1],'10'))
    circ.append(zz_xx_pump(sys[6:8],env[6:8],probs[1],'11'))
    circ.append(zz_pump(sys[8:10],env[8:9],probs[0],'11'))
    circ.append(zz_pump(sys[10:12],env[9:10],probs[1],'00'))
    circ.append(zz_pump(sys[12:14],env[10:11],probs[1],'01'))
 
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job

In [ ]:
def batch_3():
    env = [cirq.LineQubit(14),cirq.LineQubit(15),cirq.LineQubit(16),cirq.LineQubit(17),cirq.LineQubit(18),\
           cirq.LineQubit(19),cirq.LineQubit(20),cirq.LineQubit(21),cirq.LineQubit(22),cirq.LineQubit(23),\
           cirq.LineQubit(24)]
    sys = [cirq.LineQubit(0),cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(3),cirq.LineQubit(4),cirq.LineQubit(5),\
          cirq.LineQubit(6),cirq.LineQubit(7),cirq.LineQubit(8),cirq.LineQubit(9),cirq.LineQubit(10),cirq.LineQubit(11),\
          cirq.LineQubit(12),cirq.LineQubit(13)]
    circ = cirq.Circuit()
    
    circ.append(zz_xx_pump(sys[0:2],env[0:2],probs[2],'00'))
    circ.append(zz_xx_pump(sys[2:4],env[2:4],probs[2],'01'))
    circ.append(zz_xx_pump(sys[4:6],env[4:6],probs[2],'10'))
    circ.append(zz_xx_pump(sys[6:8],env[6:8],probs[2],'11'))
    circ.append(zz_pump(sys[8:10],env[8:9],probs[1],'10'))
    circ.append(zz_pump(sys[10:12],env[9:10],probs[1],'11'))
    circ.append(zz_pump(sys[12:14],env[10:11],probs[2],'00'))
 
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job

In [ ]:
def batch_4():
    env = [cirq.LineQubit(14),cirq.LineQubit(15),cirq.LineQubit(16),cirq.LineQubit(17),cirq.LineQubit(18),\
           cirq.LineQubit(19),cirq.LineQubit(20),cirq.LineQubit(21),cirq.LineQubit(22),cirq.LineQubit(23),\
           cirq.LineQubit(24)]
    sys = [cirq.LineQubit(0),cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(3),cirq.LineQubit(4),cirq.LineQubit(5),\
          cirq.LineQubit(6),cirq.LineQubit(7),cirq.LineQubit(8),cirq.LineQubit(9),cirq.LineQubit(10),cirq.LineQubit(11),\
          cirq.LineQubit(12),cirq.LineQubit(13)]
    circ = cirq.Circuit()
    
    circ.append(zz_xx_pump(sys[0:2],env[0:2],probs[3],'00'))
    circ.append(zz_xx_pump(sys[2:4],env[2:4],probs[3],'01'))
    circ.append(zz_xx_pump(sys[4:6],env[4:6],probs[3],'10'))
    circ.append(zz_xx_pump(sys[6:8],env[6:8],probs[3],'11'))
    circ.append(zz_pump(sys[8:10],env[8:9],probs[2],'01'))
    circ.append(zz_pump(sys[10:12],env[9:10],probs[2],'10'))
    circ.append(zz_pump(sys[12:14],env[10:11],probs[2],'11'))
 
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job

# probs 0:

In [6]:
qjob_1_p0= batch_1()

ValidationException: An error occurred (ValidationException) when calling the CreateQuantumTask operation (reached max retries: 4): [line 22] arbitrary unitary gates are not supported on the requested device

In [ ]:
# qjob_2_p0 = batch_2(probs[0])

In [10]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [6]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [4]:
resultsdict_1_p0 = {'0110001000111100011111111111111111': 1, '0011011000111101001111111111111111': 1, '0010011001111101001011111111111111': 1, '0110011001101001011011111111111111': 1, '0011011100111101001011111111111111': 1, '0011011100101100001011111111111111': 1, '0110011100101101001011111111111111': 1, '0011011100111100011111111111111111': 1, '0011001000111001001111111111111111': 1, '0010001101111001001011111111111111': 1, '0010011001101100011111111111111111': 1, '0110001001101001001111111111111111': 1, '0010001101111101011011111111111111': 1, '0110001001111001001111111111111111': 1, '0010001100101001011111111111111111': 1, '0111001000111101011111111111111111': 1, '0011011001101001001011111111111111': 1, '0011001100101100001011111111111111': 1, '0110011100111101011011111111111111': 1, '0111001100101100011111111111111111': 1, '0011001000101000011111111111111111': 1, '0110001001111000001011111111111111': 1, '0010001000111100011111111111111111': 1, '0111011100111100011011111111111111': 1, '0010011001101101011011111111111111': 1, '0010011001101001001011111111111111': 1, '0010011000101001001011111111111111': 1, '0011001100101101001111111111111111': 1, '0010011100101101001111111111111111': 1, '0111001101101100011111111111111111': 1, '0110001001111100001011111111111111': 1, '0011001101101101001011111111111111': 1, '0011011000111000001111111111111111': 1, '0011011000101101011011111111111111': 1, '0011011101111101011011111111111111': 1, '0011001000111101001111111111111111': 1, '0011001001101001011111111111111111': 1, '0011011100111100001011111111111111': 1, '0010001001101000001011111111111111': 1, '0011001001111101011111111111111111': 1, '0011001000111000011011111111111111': 1, '0111011000111100011011111111111111': 1, '0110011000101000011111111111111111': 1, '0011011101111100001111111111111111': 1, '0110001101111101001011111111111111': 1, '0110011101111000011111111111111111': 1, '0110011100111001001011111111111111': 1, '0110001100101001011011111111111111': 1, '0010001100111101001011111111111111': 1, '0010001100111101011011111111111111': 1, '0011001101101101001111111111111111': 1, '0010001100111101011111111111111111': 1, '0011001000101000001011111111111111': 1, '0110011000111001011111111111111111': 1, '0010011001101100011011111111111111': 1, '0111011100101100001111111111111111': 1, '0110001100101101011111111111111111': 1, '0111001001111000011111111111111111': 1, '0011011001111101011111111111111111': 1, '0111001100111101001011111111111111': 1, '0010011101101001001011111111111111': 1, '0110011001111000011011111111111111': 1, '0010011000101101001011111111111111': 1, '0111011101111000011111111111111111': 1, '0110011000101001011111111111111111': 1, '0010011000111100011111111111111111': 1, '0111001101111000011111111111111111': 1, '0011011001111001011011111111111111': 1, '0111001000101001011011111111111111': 1, '0111001100111101001111111111111111': 1, '0111011001111000001111111111111111': 1, '0011001001111001001111111111111111': 1, '0110001100111101001111111111111111': 1, '0011001000111000011111111111111111': 1, '0110001000111000011011111111111111': 1, '0110001000101100001111111111111111': 1, '0011001101101000011011111111111111': 1, '0111001000111000001111111111111111': 1, '0010001000111000001111111111111111': 1, '0010001001101000011011111111111111': 1, '0110011000111001001111111111111111': 1, '0111001100111001001011111111111111': 1, '0111001000111101011011111111111111': 1, '0011011100111000001011111111111111': 1, '0010011001111001011111111111111111': 1, '0111001000101101011011111111111111': 1, '0110001000101000001111111111111111': 1, '0111001000111100011011111111111111': 1, '0010001100101001011011111111111111': 1, '0011001000111001001011111111111111': 1, '0011001100101101001011111111111111': 1, '0111001101101101011011111111111111': 1, '0011001100101100011011111111111111': 1, '0111011001101100001111111111111111': 1, '0011011000111000011111111111111111': 1, '0110001000111000001111111111111111': 1, '0110001101101000001011111111111111': 1, '0111011101101000001111111111111111': 1, '0010001001111001011011111111111111': 1, '0010001101111000011111111111111111': 1}

In [ ]:
## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
zz00p1,zz01p1,zzxx00p1,zzxx01p1,zzxx10p1,zzxx11p1 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()

def resul_analy(resultsdict_1_p0, resultsdict_2_p0):
    for key,cnt in resultsdict_1_p0.items():
        k1 = int(key[0:2],base=2)
        k2 = int(key[2:4],base=2)
        k3 = int(key[4:6],base=2)
        zzxx00p1[k1]+=cnt
        zzxx01p1[k2]+=cnt
        zz00p1[k3]+=cnt

    for key,cnt in resultsdict_2_p0.items():
        k1 = int(key[0:2],base=2)
        k2 = int(key[2:4],base=2)
        k3 = int(key[4:6],base=2)
        zzxx10p1[k1]+=cnt
        zzxx11p1[k2]+=cnt
        zz01p1[k3]+=cnt
    
    zzxxp1 = (zzxx00p1+zzxx01p1+zzxx10p1+zzxx11p1)
    for key,cnt in zzxxp1.items():
        zzxxp1[key] = cnt/(4*repcnt)
    return zzxxp1, zz01p1, zz00p1

In [2]:
# ## ZZ Decoder
# zz10p0,zz11p0 = Counter(),Counter(),Counter(),Counter()

# for key,cnt in resultsdict10.items():
#     k1 = int(key[5:7],base=2)
#     k2 = int(key[11:13],base=2)
#     k3 = int(key[13:15],base=2)
#     k4 = int(key[15:17],base=2)
#     zz00p0[k1]+=cnt
#     zz01p0[k2]+=cnt
#     zz10p0[k3]+=cnt
#     zz11p0[k4]+=cnt
# zzp0 = (zz00p0+zz01p0+zz10p0+zz11p0)
# for key,cnt in zzp0.items():
#     zzp0[key] = cnt/(4*repcnt)
# print(zzp0)

NameError: name 'Counter' is not defined

# probs 1:

In [ ]:
# qjob_1_p1= batch_1(probs[1])

In [ ]:
# qjob_2_p1=batch_2(probs[1])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_1_p0 = {'0110001000111100011111111111111111': 1, '0011011000111101001111111111111111': 1, '0010011001111101001011111111111111': 1, '0110011001101001011011111111111111': 1, '0011011100111101001011111111111111': 1, '0011011100101100001011111111111111': 1, '0110011100101101001011111111111111': 1, '0011011100111100011111111111111111': 1, '0011001000111001001111111111111111': 1, '0010001101111001001011111111111111': 1, '0010011001101100011111111111111111': 1, '0110001001101001001111111111111111': 1, '0010001101111101011011111111111111': 1, '0110001001111001001111111111111111': 1, '0010001100101001011111111111111111': 1, '0111001000111101011111111111111111': 1, '0011011001101001001011111111111111': 1, '0011001100101100001011111111111111': 1, '0110011100111101011011111111111111': 1, '0111001100101100011111111111111111': 1, '0011001000101000011111111111111111': 1, '0110001001111000001011111111111111': 1, '0010001000111100011111111111111111': 1, '0111011100111100011011111111111111': 1, '0010011001101101011011111111111111': 1, '0010011001101001001011111111111111': 1, '0010011000101001001011111111111111': 1, '0011001100101101001111111111111111': 1, '0010011100101101001111111111111111': 1, '0111001101101100011111111111111111': 1, '0110001001111100001011111111111111': 1, '0011001101101101001011111111111111': 1, '0011011000111000001111111111111111': 1, '0011011000101101011011111111111111': 1, '0011011101111101011011111111111111': 1, '0011001000111101001111111111111111': 1, '0011001001101001011111111111111111': 1, '0011011100111100001011111111111111': 1, '0010001001101000001011111111111111': 1, '0011001001111101011111111111111111': 1, '0011001000111000011011111111111111': 1, '0111011000111100011011111111111111': 1, '0110011000101000011111111111111111': 1, '0011011101111100001111111111111111': 1, '0110001101111101001011111111111111': 1, '0110011101111000011111111111111111': 1, '0110011100111001001011111111111111': 1, '0110001100101001011011111111111111': 1, '0010001100111101001011111111111111': 1, '0010001100111101011011111111111111': 1, '0011001101101101001111111111111111': 1, '0010001100111101011111111111111111': 1, '0011001000101000001011111111111111': 1, '0110011000111001011111111111111111': 1, '0010011001101100011011111111111111': 1, '0111011100101100001111111111111111': 1, '0110001100101101011111111111111111': 1, '0111001001111000011111111111111111': 1, '0011011001111101011111111111111111': 1, '0111001100111101001011111111111111': 1, '0010011101101001001011111111111111': 1, '0110011001111000011011111111111111': 1, '0010011000101101001011111111111111': 1, '0111011101111000011111111111111111': 1, '0110011000101001011111111111111111': 1, '0010011000111100011111111111111111': 1, '0111001101111000011111111111111111': 1, '0011011001111001011011111111111111': 1, '0111001000101001011011111111111111': 1, '0111001100111101001111111111111111': 1, '0111011001111000001111111111111111': 1, '0011001001111001001111111111111111': 1, '0110001100111101001111111111111111': 1, '0011001000111000011111111111111111': 1, '0110001000111000011011111111111111': 1, '0110001000101100001111111111111111': 1, '0011001101101000011011111111111111': 1, '0111001000111000001111111111111111': 1, '0010001000111000001111111111111111': 1, '0010001001101000011011111111111111': 1, '0110011000111001001111111111111111': 1, '0111001100111001001011111111111111': 1, '0111001000111101011011111111111111': 1, '0011011100111000001011111111111111': 1, '0010011001111001011111111111111111': 1, '0111001000101101011011111111111111': 1, '0110001000101000001111111111111111': 1, '0111001000111100011011111111111111': 1, '0010001100101001011011111111111111': 1, '0011001000111001001011111111111111': 1, '0011001100101101001011111111111111': 1, '0111001101101101011011111111111111': 1, '0011001100101100011011111111111111': 1, '0111011001101100001111111111111111': 1, '0011011000111000011111111111111111': 1, '0110001000111000001111111111111111': 1, '0110001101101000001011111111111111': 1, '0111011101101000001111111111111111': 1, '0010001001111001011011111111111111': 1, '0010001101111000011111111111111111': 1}

In [ ]:
## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
zz00p1,zz01p1,zzxx00p1,zzxx01p1,zzxx10p1,zzxx11p1 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
zzxxp1, zz01p1, zz00p1 = resul_analy(resultsdict_1_p0, resultsdict_2_p0)

# probs 2:

In [ ]:
# qjob_1_p2=batch_1(probs[2])

In [ ]:
# qjob_2_p2=batch_2(probs[2])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_1_p0 = {'0110001000111100011111111111111111': 1, '0011011000111101001111111111111111': 1, '0010011001111101001011111111111111': 1, '0110011001101001011011111111111111': 1, '0011011100111101001011111111111111': 1, '0011011100101100001011111111111111': 1, '0110011100101101001011111111111111': 1, '0011011100111100011111111111111111': 1, '0011001000111001001111111111111111': 1, '0010001101111001001011111111111111': 1, '0010011001101100011111111111111111': 1, '0110001001101001001111111111111111': 1, '0010001101111101011011111111111111': 1, '0110001001111001001111111111111111': 1, '0010001100101001011111111111111111': 1, '0111001000111101011111111111111111': 1, '0011011001101001001011111111111111': 1, '0011001100101100001011111111111111': 1, '0110011100111101011011111111111111': 1, '0111001100101100011111111111111111': 1, '0011001000101000011111111111111111': 1, '0110001001111000001011111111111111': 1, '0010001000111100011111111111111111': 1, '0111011100111100011011111111111111': 1, '0010011001101101011011111111111111': 1, '0010011001101001001011111111111111': 1, '0010011000101001001011111111111111': 1, '0011001100101101001111111111111111': 1, '0010011100101101001111111111111111': 1, '0111001101101100011111111111111111': 1, '0110001001111100001011111111111111': 1, '0011001101101101001011111111111111': 1, '0011011000111000001111111111111111': 1, '0011011000101101011011111111111111': 1, '0011011101111101011011111111111111': 1, '0011001000111101001111111111111111': 1, '0011001001101001011111111111111111': 1, '0011011100111100001011111111111111': 1, '0010001001101000001011111111111111': 1, '0011001001111101011111111111111111': 1, '0011001000111000011011111111111111': 1, '0111011000111100011011111111111111': 1, '0110011000101000011111111111111111': 1, '0011011101111100001111111111111111': 1, '0110001101111101001011111111111111': 1, '0110011101111000011111111111111111': 1, '0110011100111001001011111111111111': 1, '0110001100101001011011111111111111': 1, '0010001100111101001011111111111111': 1, '0010001100111101011011111111111111': 1, '0011001101101101001111111111111111': 1, '0010001100111101011111111111111111': 1, '0011001000101000001011111111111111': 1, '0110011000111001011111111111111111': 1, '0010011001101100011011111111111111': 1, '0111011100101100001111111111111111': 1, '0110001100101101011111111111111111': 1, '0111001001111000011111111111111111': 1, '0011011001111101011111111111111111': 1, '0111001100111101001011111111111111': 1, '0010011101101001001011111111111111': 1, '0110011001111000011011111111111111': 1, '0010011000101101001011111111111111': 1, '0111011101111000011111111111111111': 1, '0110011000101001011111111111111111': 1, '0010011000111100011111111111111111': 1, '0111001101111000011111111111111111': 1, '0011011001111001011011111111111111': 1, '0111001000101001011011111111111111': 1, '0111001100111101001111111111111111': 1, '0111011001111000001111111111111111': 1, '0011001001111001001111111111111111': 1, '0110001100111101001111111111111111': 1, '0011001000111000011111111111111111': 1, '0110001000111000011011111111111111': 1, '0110001000101100001111111111111111': 1, '0011001101101000011011111111111111': 1, '0111001000111000001111111111111111': 1, '0010001000111000001111111111111111': 1, '0010001001101000011011111111111111': 1, '0110011000111001001111111111111111': 1, '0111001100111001001011111111111111': 1, '0111001000111101011011111111111111': 1, '0011011100111000001011111111111111': 1, '0010011001111001011111111111111111': 1, '0111001000101101011011111111111111': 1, '0110001000101000001111111111111111': 1, '0111001000111100011011111111111111': 1, '0010001100101001011011111111111111': 1, '0011001000111001001011111111111111': 1, '0011001100101101001011111111111111': 1, '0111001101101101011011111111111111': 1, '0011001100101100011011111111111111': 1, '0111011001101100001111111111111111': 1, '0011011000111000011111111111111111': 1, '0110001000111000001111111111111111': 1, '0110001101101000001011111111111111': 1, '0111011101101000001111111111111111': 1, '0010001001111001011011111111111111': 1, '0010001101111000011111111111111111': 1}

In [ ]:
## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
zz00p2,zz01p2,zzxx00p2,zzxx01p2,zzxx10p2,zzxx11p2 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
zzxxp2, zz01p2, zz00p2 = resul_analy(resultsdict_1_p0, resultsdict_2_p0)

# probs 3:

In [ ]:
# qjob_1_p3=batch_1(probs[3])

In [ ]:
# qjob_2_p3=batch_2(probs[3])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_1_p0 = {'0110001000111100011111111111111111': 1, '0011011000111101001111111111111111': 1, '0010011001111101001011111111111111': 1, '0110011001101001011011111111111111': 1, '0011011100111101001011111111111111': 1, '0011011100101100001011111111111111': 1, '0110011100101101001011111111111111': 1, '0011011100111100011111111111111111': 1, '0011001000111001001111111111111111': 1, '0010001101111001001011111111111111': 1, '0010011001101100011111111111111111': 1, '0110001001101001001111111111111111': 1, '0010001101111101011011111111111111': 1, '0110001001111001001111111111111111': 1, '0010001100101001011111111111111111': 1, '0111001000111101011111111111111111': 1, '0011011001101001001011111111111111': 1, '0011001100101100001011111111111111': 1, '0110011100111101011011111111111111': 1, '0111001100101100011111111111111111': 1, '0011001000101000011111111111111111': 1, '0110001001111000001011111111111111': 1, '0010001000111100011111111111111111': 1, '0111011100111100011011111111111111': 1, '0010011001101101011011111111111111': 1, '0010011001101001001011111111111111': 1, '0010011000101001001011111111111111': 1, '0011001100101101001111111111111111': 1, '0010011100101101001111111111111111': 1, '0111001101101100011111111111111111': 1, '0110001001111100001011111111111111': 1, '0011001101101101001011111111111111': 1, '0011011000111000001111111111111111': 1, '0011011000101101011011111111111111': 1, '0011011101111101011011111111111111': 1, '0011001000111101001111111111111111': 1, '0011001001101001011111111111111111': 1, '0011011100111100001011111111111111': 1, '0010001001101000001011111111111111': 1, '0011001001111101011111111111111111': 1, '0011001000111000011011111111111111': 1, '0111011000111100011011111111111111': 1, '0110011000101000011111111111111111': 1, '0011011101111100001111111111111111': 1, '0110001101111101001011111111111111': 1, '0110011101111000011111111111111111': 1, '0110011100111001001011111111111111': 1, '0110001100101001011011111111111111': 1, '0010001100111101001011111111111111': 1, '0010001100111101011011111111111111': 1, '0011001101101101001111111111111111': 1, '0010001100111101011111111111111111': 1, '0011001000101000001011111111111111': 1, '0110011000111001011111111111111111': 1, '0010011001101100011011111111111111': 1, '0111011100101100001111111111111111': 1, '0110001100101101011111111111111111': 1, '0111001001111000011111111111111111': 1, '0011011001111101011111111111111111': 1, '0111001100111101001011111111111111': 1, '0010011101101001001011111111111111': 1, '0110011001111000011011111111111111': 1, '0010011000101101001011111111111111': 1, '0111011101111000011111111111111111': 1, '0110011000101001011111111111111111': 1, '0010011000111100011111111111111111': 1, '0111001101111000011111111111111111': 1, '0011011001111001011011111111111111': 1, '0111001000101001011011111111111111': 1, '0111001100111101001111111111111111': 1, '0111011001111000001111111111111111': 1, '0011001001111001001111111111111111': 1, '0110001100111101001111111111111111': 1, '0011001000111000011111111111111111': 1, '0110001000111000011011111111111111': 1, '0110001000101100001111111111111111': 1, '0011001101101000011011111111111111': 1, '0111001000111000001111111111111111': 1, '0010001000111000001111111111111111': 1, '0010001001101000011011111111111111': 1, '0110011000111001001111111111111111': 1, '0111001100111001001011111111111111': 1, '0111001000111101011011111111111111': 1, '0011011100111000001011111111111111': 1, '0010011001111001011111111111111111': 1, '0111001000101101011011111111111111': 1, '0110001000101000001111111111111111': 1, '0111001000111100011011111111111111': 1, '0010001100101001011011111111111111': 1, '0011001000111001001011111111111111': 1, '0011001100101101001011111111111111': 1, '0111001101101101011011111111111111': 1, '0011001100101100011011111111111111': 1, '0111011001101100001111111111111111': 1, '0011011000111000011111111111111111': 1, '0110001000111000001111111111111111': 1, '0110001101101000001011111111111111': 1, '0111011101101000001111111111111111': 1, '0010001001111001011011111111111111': 1, '0010001101111000011111111111111111': 1}

In [ ]:
## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
zz00p3,zz01p3,zzxx00p3,zzxx01p3,zzxx10p3,zzxx11p3 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
zzxxp3, zz01p3, zz00p3 = resul_analy(resultsdict_1_p0, resultsdict_2_p0)

# probs 4:

In [ ]:
# qjob_1_p4=batch_1(probs[4])

In [ ]:
# qjob_2_p4=batch_2(probs[4])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_1_p0 = {'0110001000111100011111111111111111': 1, '0011011000111101001111111111111111': 1, '0010011001111101001011111111111111': 1, '0110011001101001011011111111111111': 1, '0011011100111101001011111111111111': 1, '0011011100101100001011111111111111': 1, '0110011100101101001011111111111111': 1, '0011011100111100011111111111111111': 1, '0011001000111001001111111111111111': 1, '0010001101111001001011111111111111': 1, '0010011001101100011111111111111111': 1, '0110001001101001001111111111111111': 1, '0010001101111101011011111111111111': 1, '0110001001111001001111111111111111': 1, '0010001100101001011111111111111111': 1, '0111001000111101011111111111111111': 1, '0011011001101001001011111111111111': 1, '0011001100101100001011111111111111': 1, '0110011100111101011011111111111111': 1, '0111001100101100011111111111111111': 1, '0011001000101000011111111111111111': 1, '0110001001111000001011111111111111': 1, '0010001000111100011111111111111111': 1, '0111011100111100011011111111111111': 1, '0010011001101101011011111111111111': 1, '0010011001101001001011111111111111': 1, '0010011000101001001011111111111111': 1, '0011001100101101001111111111111111': 1, '0010011100101101001111111111111111': 1, '0111001101101100011111111111111111': 1, '0110001001111100001011111111111111': 1, '0011001101101101001011111111111111': 1, '0011011000111000001111111111111111': 1, '0011011000101101011011111111111111': 1, '0011011101111101011011111111111111': 1, '0011001000111101001111111111111111': 1, '0011001001101001011111111111111111': 1, '0011011100111100001011111111111111': 1, '0010001001101000001011111111111111': 1, '0011001001111101011111111111111111': 1, '0011001000111000011011111111111111': 1, '0111011000111100011011111111111111': 1, '0110011000101000011111111111111111': 1, '0011011101111100001111111111111111': 1, '0110001101111101001011111111111111': 1, '0110011101111000011111111111111111': 1, '0110011100111001001011111111111111': 1, '0110001100101001011011111111111111': 1, '0010001100111101001011111111111111': 1, '0010001100111101011011111111111111': 1, '0011001101101101001111111111111111': 1, '0010001100111101011111111111111111': 1, '0011001000101000001011111111111111': 1, '0110011000111001011111111111111111': 1, '0010011001101100011011111111111111': 1, '0111011100101100001111111111111111': 1, '0110001100101101011111111111111111': 1, '0111001001111000011111111111111111': 1, '0011011001111101011111111111111111': 1, '0111001100111101001011111111111111': 1, '0010011101101001001011111111111111': 1, '0110011001111000011011111111111111': 1, '0010011000101101001011111111111111': 1, '0111011101111000011111111111111111': 1, '0110011000101001011111111111111111': 1, '0010011000111100011111111111111111': 1, '0111001101111000011111111111111111': 1, '0011011001111001011011111111111111': 1, '0111001000101001011011111111111111': 1, '0111001100111101001111111111111111': 1, '0111011001111000001111111111111111': 1, '0011001001111001001111111111111111': 1, '0110001100111101001111111111111111': 1, '0011001000111000011111111111111111': 1, '0110001000111000011011111111111111': 1, '0110001000101100001111111111111111': 1, '0011001101101000011011111111111111': 1, '0111001000111000001111111111111111': 1, '0010001000111000001111111111111111': 1, '0010001001101000011011111111111111': 1, '0110011000111001001111111111111111': 1, '0111001100111001001011111111111111': 1, '0111001000111101011011111111111111': 1, '0011011100111000001011111111111111': 1, '0010011001111001011111111111111111': 1, '0111001000101101011011111111111111': 1, '0110001000101000001111111111111111': 1, '0111001000111100011011111111111111': 1, '0010001100101001011011111111111111': 1, '0011001000111001001011111111111111': 1, '0011001100101101001011111111111111': 1, '0111001101101101011011111111111111': 1, '0011001100101100011011111111111111': 1, '0111011001101100001111111111111111': 1, '0011011000111000011111111111111111': 1, '0110001000111000001111111111111111': 1, '0110001101101000001011111111111111': 1, '0111011101101000001111111111111111': 1, '0010001001111001011011111111111111': 1, '0010001101111000011111111111111111': 1}

In [ ]:
## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
zz00p4,zz01p4,zzxx00p4,zzxx01p4,zzxx10p4,zzxx11p4 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
zzxxp4, zz01p4, zz00p4 = resul_analy(resultsdict_1_p0, resultsdict_2_p0)

In [12]:
# ## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
# zz00p0,zz01p0,zzxx00p0,zzxx01p0,zzxx10p0,zzxx11p0 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()

# for key,cnt in resultsdict_1_p0.items():
#     k1 = int(key[0:2],base=2)
#     k2 = int(key[2:4],base=2)
#     k3 = int(key[4:5],base=2)
#     zzxx00p0[k1]+=cnt
#     zzxx01p0[k2]+=cnt
#     zz00p0[k3]+=cnt

# for key,cont in resultsdict_2_p0.items():
#     k1 = int(key[0:2],base=2)
#     k2 = int(key[2:4],base=2)
#     k3 = int(key[4:5],base=2)
#     zzxx10p0[k1]+=cnt
#     zzxx11p0[k2]+=cnt
#     zz01p0[k3]+=cnt
    
# zzxxp0 = (zzxx00p0+zzxx01p0+zzxx10p0+zzxx11p0)
# for key,cnt in zzxxp0.items():
#     zzxxp0[key] = cnt/(4*repcnt)
# print(zzxxp0)

Counter({1: 0.285, 0: 0.26, 2: 0.24, 3: 0.215})
